In [ ]:
# Step 1. Imports
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import ta
import os
from datetime import datetime



In [ ]:


# Step 2. Parameters
TICKER = "AAPL"
START_DATE = "2015-01-01"
END_DATE = datetime.today().strftime('%Y-%m-%d')




In [ ]:
# Step 3. Create data folders if not exist
os.makedirs("../data/raw", exist_ok=True)
os.makedirs("../data/processed", exist_ok=True)



In [ ]:
# Step 4. Download historical data
print(f"Downloading {TICKER} data...")
data = yf.download(TICKER, start=START_DATE, end=END_DATE)
data.reset_index(inplace=True)
print(f"Data shape: {data.shape}")
data.head()



In [ ]:
# Step 5. Save raw data
raw_path = f"../data/raw/{TICKER}.csv"
data.to_csv(raw_path, index=False)
print(f"Raw data saved to {raw_path}")



In [ ]:
# Step 6. Add technical indicators using 'ta' library
df = data.copy()

# Simple Moving Average (SMA)
df["SMA_20"] = ta.trend.sma_indicator(df["Close"], window=20)
df["SMA_50"] = ta.trend.sma_indicator(df["Close"], window=50)

# Relative Strength Index (RSI)
df["RSI_14"] = ta.momentum.rsi(df["Close"], window=14)

# Bollinger Bands
boll = ta.volatility.BollingerBands(df["Close"], window=20, window_dev=2)
df["BB_High"] = boll.bollinger_hband()
df["BB_Low"] = boll.bollinger_lband()
df["BB_Width"] = df["BB_High"] - df["BB_Low"]



In [ ]:
# Step 7. Drop NaN values
df.dropna(inplace=True)



In [ ]:
# Step 8. Save processed data
processed_path = f"../data/processed/{TICKER}_features.csv"
df.to_csv(processed_path, index=False)
print(f"Processed data saved to {processed_path}")



In [ ]:
# Step 9. Quick visualization


plt.figure(figsize=(12,6))
plt.plot(df["Date"], df["Close"], label="Close Price", color="blue")
plt.plot(df["Date"], df["SMA_20"], label="SMA 20", color="orange")
plt.plot(df["Date"], df["SMA_50"], label="SMA 50", color="green")
plt.fill_between(df["Date"], df["BB_Low"], df["BB_High"], color='gray', alpha=0.1, label="Bollinger Band")
plt.title(f"{TICKER} Price with Technical Indicators")
plt.legend()
plt.show()